In [1]:
import pandas as pd

In [2]:
playerstats = pd.read_csv("AllPlayerStats.csv", low_memory=False)

In [3]:
playerstats = playerstats.rename(columns={'Unnamed: 36': 'Season'})

In [4]:
playerstats=playerstats[["Player","Pos", "Squad", "MP","Min", "Gls", "Ast","G+A", "PK", "Season"]]

In [5]:
table = pd.read_csv("PremTables.csv")
pots = pd.read_csv("POTSWinner.csv")

In [6]:
table[["Notes"]] = table[["Notes"]].fillna("No notes")

In [7]:
playerstats = playerstats[~playerstats["MP"].str.contains("MP")] #it excludes the row with the parameters given in contain 
                                                                 #if the column you give first has a row with the parameter provided in contains it deletes that row

In [8]:
table = table[["Rk", "Squad", "W", "D", "L","GF","GA", "Pts", "Season"]]

In [9]:
combined = playerstats.merge(table, how="outer", on= ["Squad", "Season"])
combined = combined.merge(pots, how = "outer", on=["Squad", "Player", "Season"])
del combined["Unnamed: 0"]
del combined["Nationality"]

In [10]:
#outer es para que se haga merge al final de la tabla que ya tenias, y el otro es para que combine el squad y año en el que paso la temporada con el jugador. 
combined = combined.apply(pd.to_numeric, errors="ignore")

C:\Users\Jorge HMV\AppData\Local\Temp\ipykernel_45504\249434140.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  combined = combined.apply(pd.to_numeric, errors="ignore")


In [11]:
#en machine learning es mas facil trabajar con valores numericos asi que convertimos a numero aquello que se pueda convertir
#combined.dtypes #nos sirve para checar que se cambio y que no

In [12]:
combined.to_csv("PremierPOTS.csv")

In [13]:
goalScorer = combined[combined["MP"]>30].sort_values("Gls", ascending=False).head(10) #acomoda la informacion para que salgan los que han jugado mas de 30 partidos y lo acomoda por quien metio mas goles

In [14]:
combined = combined.fillna(0)

In [15]:
#pd.isnull(combined).sum() #check if there is null values

In [16]:
#combined[pd.isnull(combined["Gls"])]

In [17]:
#highest_GA = combined.groupby("Season").apply(lambda x: x.sort_values("G+A", ascending= False).head(1))

In [18]:
#winners = combined.groupby("Season").apply(lambda x: x.sort_values("Winner", ascending= False).head(1)) #sort by winner and season

In [19]:
combined.columns

Index(['Player', 'Pos', 'Squad', 'MP', 'Min', 'Gls', 'Ast', 'G+A', 'PK',
       'Season', 'Rk', 'W', 'D', 'L', 'GF', 'GA', 'Pts', 'Winner'],
      dtype='object')

In [20]:
predictors = [ 'MP', 'Min', 'Gls', 'Ast', 'G+A', 'PK', 'Season',
       'Rk', 'W', 'D', 'L','GF', 'GA', 'Pts']

In [21]:
train = combined[(combined["Season"] < 2022) & (combined["Season"] > 1994)]

In [22]:
test = combined[combined["Season"] == 2022]

In [23]:
from sklearn.linear_model import Ridge

In [24]:
reg = Ridge(alpha=.1)
reg.fit(train[predictors], train["Winner"])
predictions = reg.predict(test[predictors]) #entrenas al modelo con la informacion de train poniendolo a prueba en test

In [25]:
predictions = pd.DataFrame(predictions, columns =["Predictions"], index = test.index) #hacer mas leible el formato con pandas

In [26]:
combination = pd.concat([test[["Player","Winner"]], predictions], axis=1)

In [27]:
combination = combination.sort_values("Winner", ascending=False)
combination["Rk"] = list(range(1, combination.shape[0]+1))

In [28]:
combination = combination.sort_values("Predictions", ascending=False)
combination["PredictedRk"] = list(range(1, combination.shape[0]+1))

In [41]:
def find_ap(df): #average precision
    actual = df.sort_values("Winner", ascending=False).head(1)
    predicted = df.sort_values("Predictions", ascending=False)
    ps=[]
    found=0
    seen=1
    for index, row in predicted.iterrows():
        if row["Player"] in actual ["Player"].values:
            found +=1
            ps.append(found/seen)
        seen+=1
    return sum(ps)/ len(ps) #error metric

In [30]:
years= list(range(1995,2023))
aps = []
allpredictions = []
for year in years[5:]: #entrenas a los modelos y vas incrementando el año cada vez
    
    train = combined[(combined["Season"] < year) & (combined["Season"] > 1994)]
    test = combined[combined["Season"] == year]
    
    reg = Ridge(alpha=.1)
    reg.fit(train[predictors], train["Winner"])
    predictions = reg.predict(test[predictors]) 

    predictions = pd.DataFrame(predictions, columns =["Predictions"], index = test.index)

    combination = pd.concat([test[["Player","Winner"]], predictions], axis=1)
    
    allpredictions.append(combination)
    aps.append(find_ap(combination))

In [31]:
def addranks(df):
    df = df.sort_values("Winner", ascending=False)
    df['Rk'] = list(range(1, df.shape[0]+1))
    df = df.sort_values("Predictions", ascending=False)
    df['PredictedRK'] = list(range(1, df.shape[0]+1))
    df['Diff'] = df["Rk"] - df["PredictedRK"]
    return df

In [32]:
#add_ranks(all_predictions[12]).sort_values("Winner", ascending=False)

In [33]:
sum(aps)/len(aps)

0.46418627203859764

In [34]:
addranks(allpredictions[1]).sort_values("Winner", ascending=False)

,Player,Winner,Predictions,Rk,PredictedRK,Diff
631,Patrick Vieira,1.0,-0.013448,1,74,-73
14184,Kevin Phillips,0.0,0.005843,484,4,480
10411,Alen Bokšić,0.0,0.004441,388,5,383
807,Thierry Henry,0.0,0.003583,24,6,18
7358,Mark Viduka,0.0,0.001478,264,7,257
...,...,...,...,...,...,...
7509,Andy Impey,0.0,-0.024721,275,555,-280
13637,Uwe Rösler,0.0,-0.024817,467,556,-89
7877,Steve Guppy,0.0,-0.025145,298,557,-259
11267,Lomana LuaLua,0.0,-0.025477,433,558,-125


In [57]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

In [60]:
def backtest(df, model, years, predictors):
    aps = []
    allpredictions = []
    for year in years: #entrenas a los modelos y vas incrementando el año cada vez
        
        train = df[(df["Season"] < year) & (df["Season"] > 1994)]
        test = df[df["Season"] == year]
        
        model.fit(train[predictors], train["Winner"])
        predictions = model.predict(test[predictors]) 
    
        predictions = pd.DataFrame(predictions, columns =["Predictions"], index = test.index)  #crea la columna y pone adentro las predicciones
        
        combination = pd.concat([test[["Player","Winner"]], predictions], axis=1) #junta el df del año en que se testeo con las predicciones, juntando 
        combination = combination = addranks(combination)
        
        allpredictions.append(combination)
        aps.append(find_ap(combination))
    return sum(aps)/len(aps), aps, pd.concat(allpredictions)

In [43]:
mean_ap, aps, allpredictions = backtest(combined, reg, years[5:], predictors)

In [45]:
combined["NPos"] = combined["Pos"].astype("category").cat.codes

In [47]:
combined["NSquad"] = combined["Squad"].astype("category").cat.codes

In [48]:
combined.head()

,Player,Pos,Squad,MP,Min,Gls,Ast,G+A,PK,Season,Rk,W,D,L,GF,GA,Pts,Winner,NPos,NSquad
0,Aaron Ramsdale,GK,Arsenal,34.0,3060.0,0.0,0.0,0.0,0.0,2022,5.0,22.0,3.0,13.0,61.0,48.0,69.0,0.0,7,0
1,Aaron Ramsdale,GK,Arsenal,38.0,3420.0,0.0,0.0,0.0,0.0,2023,2.0,26.0,6.0,6.0,88.0,43.0,84.0,0.0,7,0
2,Aaron Ramsdale,GK,Arsenal,6.0,540.0,0.0,0.0,0.0,0.0,2024,2.0,28.0,5.0,5.0,91.0,29.0,89.0,0.0,7,0
3,Aaron Ramsey,MF,Arsenal,9.0,230.0,0.0,1.0,1.0,0.0,2009,4.0,20.0,12.0,6.0,68.0,37.0,72.0,0.0,8,0
4,Aaron Ramsey,MF,Arsenal,18.0,786.0,3.0,3.0,6.0,0.0,2010,3.0,23.0,6.0,9.0,83.0,41.0,75.0,0.0,8,0


In [50]:
from sklearn.ensemble import RandomForestRegressor

In [68]:
rf = RandomForestRegressor(n_estimators=50, random_state=1, min_samples_split=5) #decision trees, if you run twice get the same result, the more accurate it will be the lower it is
mean_ap, aps, allpredictions = backtest(combined, rf, years[5:], predictors + ["NPos","NSquad"])

In [69]:
mean_ap

0.39738903792702607

In [70]:
mean_ap, aps, allpredictions = backtest(combined, reg, years[5:], predictors)

In [71]:
mean_ap

0.46418627203859764